# Fully customized spaCy Tokenizer Pipleines
- spaCy의 nlp에 대하여 가장 중요한 역할을 하는 것이 바로 tokenization이다.
- spaCy는 규칙 기반의 tokenization 방식을 지원하지만, 기본적으로 라틴어에 적합한 형태임.
- 그러므로, 아시아권 언어에 대해서는 각 국가별로 이미 만들어진 tokenizer를 활용하는 것이 좋음.

---

## nlp pipeline에 토크나이저 wrapping하기.
- spacy 한국어의 경우 mecab과 natto를 백본으로 만들어짐.
- Domain-specific하게 학습한 soynlp tokenizer로 spacy tokenizer를 wrapping할 것임.

### 커스텀 토크나이저 정의하기.
#### callable object로 만들자.
- input : text
- output : Doc object

#### `nlp.vocab`을 넘겨주자.
- 언어별 기본 nlp를 넘겨주자.

#### custom variable
- 형태소 분석기반 토크나이저
    - spacytokenizer 커스텀
- 커스텀 품사 정의
    - tokenizer 커스텀
    - spacytokenizer 커스텀

## 예시
- soynlp 같은 경우 품사가 존재하지 않기 때문에 후처리로 L과 R을 처리하는 객체 정의함. (`pipeline/ko.py`확인)
- SpacyTokenizer에 기 품사 또는 커스텀 품사를 attribute로 넘겨받기 위해서 Attribute extensions(`._.something`)을 활용함.
    - 오피셜하게는 tag attribute에 wrapping하는 것이 맞는 방법이나, 간단하게 attribute extensions을 활용하자.

- **Define custom attribute globally**

In [1]:
from spacy.tokens import Token
Token.set_extension("tag_", default=None)

- **Define and initialize tokenizer and SpacyTokenizer**

In [2]:
from spacy.lang.ko import Korean
from pipeline import ko

nlp = Korean()
nlp.tokenizer = ko.SpacyTokenizer(nlp.vocab)

In [4]:
for token in nlp('요새 코로나 때문에 마스크를 많이 쓰는 사람들은 피부가 더 거칠어지고 건조하고 자극을 많이받아서 피부 트러블도 많아 지는 시점이구요!'):
    print(token.text, token._.tag_)

요새 L
코로나 L
때문 L
에 R
마스크 L
를 R
많이 L
쓰는 L
사람들 L
은 R
피부 L
가 R
더 L
거칠어지고 L
건조 L
하고 R
자극 L
을 R
많이받아서 L
피부 L
트러블 L
도 R
많아 L
지는 L
시점 L
이구요 R


---

## nlp pipline with text in tabular data
- tabular data에 포함된 텍스트 데이터를 잘 분석하기 위해서는 데이터프레임과 spacy object 간의 자유로운 transform이 가능해야된다.
- **df $\longrightarrow$ spacy $\longrightarrow$ df**와 같은 일련의 파이프라인을 구축하는 것이 중요하다.

### from `df` to `spacy`

- **tabular 데이터를 로드하고, spacy 변수 정의**

In [1]:
import pandas as pd
df = pd.read_csv('data/my_data.csv')
print(df.shape)
df.head()

(1500, 3)


,Id,Mention Title,Mention Content
0,360101-102055396016,Sulwhasoo,정답 : 쉬어 래스팅 이아름 님에게 알립니다.
1,360101-141133562509,[랑콤] 수지가 소개하는 리얼 메이크업 팁! Suzy's Real Make-up t...,천사 수지❤️
2,360101-92181315780,NaN,"#에스티로더 39컬러 NEW 신상립! #엔비립페인트 지속력이 을매나 짱짱헌지,,,,..."
3,360101-133213819477,김선아 공항패션 스타일링 글쓴이 보배 등록일 2019-11-04 15:24 조회수 ...,전체보기 일상 부부 시집/친정 육아 직장살림 좋은글 웃음 고민/익명 연예/시사 내가...
4,360101-74466121153,NaN,#랑콤#립스틱#박젼님선물🌺고마워용


In [2]:
from spacy.tokens import Doc, Token
from spacy.lang.ko import Korean
from pipeline import ko

In [3]:
## set custom attributes
# tabular 데이터에서 document의 feature들을 doc attribute로 넘길 수 있음.
Doc.set_extension('Id', default=None)
Doc.set_extension('Mention', default=None)

Token.set_extension('tag_', default=None) 

In [4]:
## wraaping custom tokenizer
nlp = Korean()
nlp.tokenizer = ko.SpacyTokenizer(nlp.vocab)

- **Context passing을 활용하자.**
    - dataframe $\longrightarrow$ list of tuple (str, dict)
    - 여기서 dictionary에 context 즉, 각 문서(document)에 대응하는 feature들을 context형태로 넘기자.
    - 본 예제에서는 3000개의 document에 대하여 attribute로 **id값**과 **title여부**를 context로 넘길 것이다.

In [5]:
df = df.fillna('') # 결측값 pd.NA -> ''
data = df.to_dict('r')
mention_title = [(doc['Mention Title'], {'Id':doc['Id'], 'Mention':'Mention Title'}) 
                 for doc in data if (doc['Mention Title']!=None) and (doc['Mention Title'].strip() != '')]
mention_content = [(doc['Mention Content'], {'Id':doc['Id'], 'Mention':'Mention Content'}) 
                   for doc in data if (doc['Mention Content']!=None) and (doc['Mention Content'].strip() != '')]

In [6]:
print(len(mention_title), len(mention_content))

1114 1490


- **nlp.pipe와 doc_bin을 통해 효율적인 파이프라인 구성.**
    - doc_bin의 doc객체를 좀 더 메모리 효율적으로 관리함.

In [7]:
from spacy.tokens import DocBin
doc_bin = DocBin(store_user_data=True)

In [8]:
total = mention_title + mention_content
for doc, context in nlp.pipe(total, as_tuples=True, n_process=1): # process=1에서 에러가 없을 경우, multiprocess로 확장.
    doc._.Id = context['Id']
    doc._.Mention = context['Mention']
    doc_bin.add(doc)

- **get doc**
    - __len__이 정의되어 있음.
    - `get_docs(nlp.vocab)`을 통해 doc들의 generator로 만들 수 있음.

In [9]:
len(doc_bin)

2604

In [10]:
for doc in doc_bin.get_docs(nlp.vocab):
    print(doc._.Id, doc._.Mention)
    for token in doc:
        print(token.text, token._.tag_, end=' ')
    break

360101-102055396016 Mention Title
sulwhasoo L 

### from `spacy` to `df`

In [11]:
# mention title과 mention content를 각각 df로 만들기.
doc_gen = doc_bin.get_docs(nlp.vocab)

mention_title = []
mention_content = []
for doc in doc_gen:
    if doc._.Mention == 'Mention Title':
        mention_title.append({'Id':doc._.Id, doc._.Mention:doc.text})
        continue
    mention_content.append({'Id':doc._.Id, doc._.Mention:doc.text})

- outer join으로 합치면 끝

In [12]:
result = pd.DataFrame(mention_content).merge(pd.DataFrame(mention_title), how='outer')
result.shape

(1500, 3)

---

## Save and Load the Doc

- `nlp`와 `doc_bin`을 byte화 한다.
- `pickle`로 저장한다.

### Serializing

In [13]:
import pickle

In [18]:
with open('data/nlp.pkl', 'wb') as f:
    pickle.dump(nlp.to_bytes(), f)
    
with open('data/doc.pkl', 'wb') as f:
    pickle.dump(doc_bin.to_bytes(), f)

### Deserializing

In [1]:
import pickle
import spacy
from spacy.tokens import Doc, Token
from spacy.tokens import DocBin

# custom attributes you can access by _
Doc.set_extension('Id', default=None)
Doc.set_extension('Mention', default=None)

Token.set_extension('tag_', default=None) 

- **load nlp**

In [2]:
nlp = spacy.blank('ko')
with open('data/nlp.pkl', 'rb') as f:
    nlp.from_bytes(pickle.load(f))

- **load docs**

In [3]:
with open('data/doc.pkl', 'rb') as f:
    doc_bin = DocBin(store_user_data=True).from_bytes(pickle.load(f))

In [6]:
docs = list(doc_bin.get_docs(nlp.vocab))

In [7]:
print(docs[113])
print(len(docs))

[1212타임] 설화수 자음 2종세트 2018년 (쇼핑백 포함)
2604
